At the top of every petrify project is usually the same list of imports:

In [1]:
from petrify.space import Vector, Point, Plane
from petrify import tau, plane, render

Of course, the humble box is the literal building block of many pieces of solid geometry (you can use the mouse to interact with the camera in each of these examples):

In [2]:
from petrify.solid import Box

render(Box(Point.origin, Vector(1, 1, 1)))

Preview(child=BufferGeometry(attributes={'position': <BufferAttribute shape=(36, 3), dtype=float32>, 'normal':…

And its round cousin, the cylinder, is also frequently useful:

In [3]:
from petrify.solid import Cylinder

render(Cylinder(Point.origin, Vector.basis.y * 3, 1.0, segments=20))

Preview(child=BufferGeometry(attributes={'position': <BufferAttribute shape=(228, 3), dtype=float32>, 'normal'…

Another method for forming simple solids is to extrude a planar shape along a simple vector:

In [4]:
from petrify.solid import Basis, PlanarPolygon, PolygonExtrusion

triangle = plane.Polygon([
    plane.Point(0, 0),
    plane.Point(0, 2),
    plane.Point(1, 1)
])

render(PolygonExtrusion(PlanarPolygon(Basis.unit, triangle), Vector(0, 0, 1)))

Preview(child=BufferGeometry(attributes={'position': <BufferAttribute shape=(24, 3), dtype=float32>, 'normal':…

There are many advanced methods for spinning, extruding, and chaining planar polygons into complex solids. They are covered in detail in [another notebook](advanced.ipynb).

Finally, perhaps the most common method for forming solids is the union of many smaller parts:

In [7]:
import random
from petrify.solid import Union

def spin(shape):
    return (
        shape
            .rotate_around(random.uniform(0, tau / 16), Vector.basis.x)
            .rotate_around(random.uniform(0, tau / 16), Vector.basis.y)
            .rotate_around(random.uniform(0, tau / 16), Vector.basis.z)
    )

def delta(): return random.uniform(0.75, 1.25)

cubes = [
    spin(Box(
        Point.origin,
        random.uniform(1, 2) * Vector(1, 1, 1)
    )) + Point(x * delta(), y * delta(), delta())
    for x in range(8)
    for y in range(3)
]

render(Union(cubes))

Preview(child=BufferGeometry(attributes={'position': <BufferAttribute shape=(19488, 3), dtype=float32>, 'norma…

This is a powerful feature which is covered in more detail in its [own example notebook](csg.ipynb).